<a href="https://colab.research.google.com/github/Jarcos09/Tareas/blob/main/docling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
from docling.document_converter import DocumentConverter

import pandas as pd
import os

In [115]:
from huggingface_hub import login


In [116]:
# --- Configuración ---
# Puedes usar una URL o una ruta a un archivo PDF local.
# Para este ejemplo, usaré un PDF de ejemplo que podrías reemplazar.
# Si tienes un PDF local, asegúrate de que la ruta sea correcta.

# pdf_source = "ruta/a/tu/documento.pdf" # Descomenta y usa tu propio PDF local
pdf_source = os.path.join('Documentos', 'Prueba.pdf')

# --- Inicializar el conversor de Docling ---
converter = DocumentConverter()

print(f"Procesando el PDF desde: {pdf_source}")

Procesando el PDF desde: Documentos/Prueba.pdf


In [117]:
def imprime_tabla(tabla_data, num_rows, num_cols):

  table_matrix = [[None for _ in range(num_cols)] for _ in range(num_rows)]

  # 3. Llenar la matriz con el texto de las celdas
  for cell in tabla_data.table_cells:
      row_start = cell.start_row_offset_idx
      col_start = cell.start_col_offset_idx
      row_span = cell.row_span
      col_span = cell.col_span
      text = cell.text

      # Asignar el texto a la posición correcta, considerando row_span y col_span
      # Aunque en tus datos todos son span 1, este código es más robusto.
      for r_offset in range(row_span):
          for c_offset in range(col_span):
              current_row = row_start + r_offset
              current_col = col_start + c_offset
              if 0 <= current_row < num_rows and 0 <= current_col < num_cols:
                  table_matrix[current_row][current_col] = text
              else:
                  print(f"Advertencia: Celda fuera de límites calculados: {cell}")

  # 4. Identificar los nombres de las columnas (headers)
  column_names = [None] * num_cols
  for cell in tabla_data.table_cells:
      if cell.column_header:
          # Asume que los encabezados están en el primer row_offset_idx de los headers
          if cell.start_row_offset_idx == 0:
              col_idx = cell.start_col_offset_idx
              if 0 <= col_idx < num_cols:
                  column_names[col_idx] = cell.text

  # Si algunos nombres de columna no se encontraron o son None (por ejemplo, si la primera fila no tuviera encabezados para todas las columnas), puedes usar nombres genéricos
  for i, name in enumerate(column_names):
      if name is None:
          column_names[i] = f'Columna_{i}'

  # 5. Crear el DataFrame de pandas
  # Eliminar la fila de encabezados de la matriz si los encabezados se usarán como nombres de columna
  # En tu caso, los encabezados están en la fila 0 (start_row_offset_idx: 0)
  # Así que los datos "reales" de la tabla empiezan desde la fila 1.
  data_rows = table_matrix[1:] # Excluye la primera fila que son los headers

  df = pd.DataFrame(data_rows, columns=column_names)

  # Opcional: Si la primera columna 'No' es un índice y no un dato, puedes establecerla como índice
  # Si es un dato que quieres conservar, simplemente omite esta línea.
  if 'Columna_0' in df.columns:
      df = df.set_index('Columna_0')

  # Mostrar el DataFrame
  print("\nDataFrame generado:")
  print(df)

  #df.to_csv("prueba.csv", index=True, encoding="UTF-8")

In [123]:
try:
    result = converter.convert(pdf_source)
    doc = result.document

    print("\n--- Información General del Documento ---")
    dict_repr = doc.export_to_dict()
    num_texts = len(dict_repr.get('texts', []))
    num_tables = len(dict_repr.get('tables', []))

    print(f"Elementos de texto encontrados: {num_texts}")
    print(f"Tablas encontradas: {num_tables}")

except Exception as e:
    print(f"Ocurrió un error al procesar el documento: {e}")


--- Información General del Documento ---
Elementos de texto encontrados: 74
Tablas encontradas: 3


In [124]:
 # --- Extraer y mostrar los primeros párrafos ---
print("\n--- Párrafos ---")
# Itera sobre los elementos del documento para encontrar párrafos.
# Docling organiza el contenido en una estructura jerárquica.
for item, _ in doc.iterate_items():
  if item.label == "text":
    palabras = item.text.split()
    lineas = [' '.join(palabras[i:i+10]) for i in range(0, len(palabras), 10)]
    parrafo_formateado = '\n'.join(lineas)
    print(f"• {parrafo_formateado}\n")





--- Párrafos ---
• de Operación Administrativa Desconcentrada Regional en Tamaulipas SUBDELEGACION NUEVO LAREDO
DEPARTAMENTO DE AFILIACION Órgano

• LIC. JORGE AUGUSTO RIVAS BRES ABOGADO ÁREA JURIDICA

• En seguimiento al oficio circular N'0952 17 9200/203 de fecha
02 de Septiembre del 2022, girado por el Luis Gerardo
Magaña Zaga Titular de la Unidad de Incorporación al Seguro
Social; en el que se instruye a los Jefes de
Departamento de Afiliación Vigencia de Subdelegaciones que integran esta OOAD
Regional Tamaulipas, conforme identificar mensualmente a aquellos patrones marcados con
causa 50 relacionados dentro del proceso interactivo AFIL-16 que resulte
de la ejecución del Proceso Mensual de Patrones Vigentes Sin
Trabajadores y Patrones Asociados por Número de Relación de Empresa
Municipio 0 Primeras Ocho Posiciones de acuerdo a las fechas
establecidas en la sexta columna del calendario RT8-110. Mtro. las

• Por lo que una vez analizada la totalidad de la
información, se solicita si

In [125]:
print("\n--- Tablas ---")

for item, _ in doc.iterate_items():
    if item.label == "table":

        max_row_idx = 0
        max_col_idx = 0

        for cell in item.data.table_cells:
            if cell.end_row_offset_idx > max_row_idx:
                max_row_idx = cell.end_row_offset_idx
            if cell.end_col_offset_idx > max_col_idx:
                max_col_idx = cell.end_col_offset_idx

        num_rows = max_row_idx
        num_cols = max_col_idx

        print(f"\n• La tabla tiene {num_rows} renglones y {num_cols} columnas.")
        imprime_tabla(item.data, num_rows, num_cols)


--- Tablas ---

• La tabla tiene 21 renglones y 3 columnas.

DataFrame generado:
     No. REGISIRO PAIRONAL                                  PAIRON
0   None       E9713201106                   LORENZO GARCIA RANGEL
1   None       E9716452102              RENE RAYMUNDO GUERRA SILVA
2      3       E9718175107               MARIA DE JESUS MU#OZ LUGO
3   None       E9724796102                  GABRIEL VIELMA RAMIREZ
4      5       E9730880106                   SERGIO HERRERA RAFAEL
5      6       E9738651103                  JOSEFINA AVILA DELGADO
6   None       E9739195100             GILBERTO RODRIGUEZ GONZALEZ
7      8       E9739404106             MAGDALENA BRISE#O RODRIGUEZ
8      9       E9739425101       JUANA GUADALUPE BARRAGAN MARTINEZ
9   None       E9739441108           SANDRA ABIGAIL MARTINEZ BASTO
10  None       E9739618101            LAURA YOLANDA ALVARADO REYES
11  None       E9740493106                   HECTOR MOLANO ARIZOLA
12    13       E9740987107                  EFR